In [1]:
def success_metric(supply,forecast):
    return round(abs(forecast - supply).sum() / (supply.mean(axis = 0) * len(supply)),10)

In [13]:
def plot_heatmap(x,y,z,center=float,cmap='coolwarm'):
    df = pd.DataFrame.from_dict(np.array([x,y,z]).T)
    df.columns = ['X_coordinate','Y_coordinate','max_supply']
    df['max_supply'] = pd.to_numeric(df['max_supply'])

    pivotted= df.pivot('X_coordinate','Y_coordinate','max_supply')
    pivotted
    plt.figure(figsize=(12, 9))
    sns.heatmap(pivotted,annot=True,cmap=cmap,center=center)

In [9]:
def calculate_prophet_forecast(df_dict_zones, cut_date):
    ResultDictProphet = {elem : pd.DataFrame for elem in zone_names}

    for key, value in df_dict_zones.items():

        data = value[['timestamp','supply']]
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data.set_index('timestamp', inplace=True)

        data_hours = data.reset_index()
        data_hours.columns = ['ds','y']
        data_hours = data_hours.sort_values(by=['ds'])

        train = data_hours[ data_hours.ds < cut_date ]
        test = data_hours[ data_hours.ds >= cut_date ]

        prophet = Prophet()
        prophet.fit(train)
        future = prophet.make_future_dataframe(periods=len(test), freq='H', include_history=False)
        forecast = prophet.predict(future)
        ResultDictProphet[key] = success_metric(test.y.values,forecast.yhat.values)
    return ResultDictProphet

In [10]:
def calculate_naive_forecast(df_dict_zones, cut_date):
    ResultDictNaive = {elem : pd.DataFrame for elem in zone_names}

    for key, value in df_dict_zones.items():

        data = value[['timestamp','supply']]
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data.set_index('timestamp', inplace=True)

        data_hours = data.reset_index()
        data_hours.columns = ['ds','y']
        data_hours = data_hours.sort_values(by=['ds'])
        test = data_hours[ data_hours.ds >= cut_date ]
        
        test_data = value[ value['timestamp'] >= cut_date ]
        naive_forecast = test_data['forecast']
        ResultDictNaive[key] = success_metric(test.y.values,naive_forecast.values)
    return ResultDictNaive

In [11]:
def prepare_xyz(ResultDict):
    zone_results = pd.DataFrame.from_dict(ResultDict, orient='index')
    zone_results = zone_results.reset_index()
    first_column = zone_results.iloc[:, 0] 
    zone_results['x_coordinate'] = zone_results.apply (lambda row: row['index'][:1], axis=1)
    zone_results['y_coordinate'] = zone_results.apply (lambda row: row['index'][2:3], axis=1)
    return [zone_results['x_coordinate'],zone_results['y_coordinate'],zone_results[0]]